# Implementar exemplos de Redes Neurais usando PyTorch, variando hiperparâmetros de treinamento.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_moons # Para gerar dados não lineares
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("--- Iniciando Exemplos de Redes Neurais com Variação de Hiperparâmetros ---")

# 1. Gerar Dados Sintéticos para Classificação Binária (Exemplo: Luas)
# Usaremos make_moons para criar um conjunto de dados não linear, ideal para redes neurais
X, y = make_moons(n_samples=200, noise=0.15, random_state=42)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1) # Redimensionar para (N, 1)

# Dividir dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Visualizar os dados gerados
plt.figure(figsize=(8, 6))
plt.scatter(X_train[:, 0].numpy(), X_train[:, 1].numpy(), c=y_train.numpy().flatten(), cmap='viridis', label='Dados de Treinamento')
plt.scatter(X_test[:, 0].numpy(), X_test[:, 1].numpy(), c=y_test.numpy().flatten(), cmap='plasma', marker='x', s=100, label='Dados de Teste')
plt.title('Dados Sintéticos de Classificação (Luas)')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.legend()
plt.grid(True)
plt.show()

# 2. Definir o Modelo da Rede Neural
class SimpleNN(nn.Module):
    def __init__(self, activation_fn=nn.ReLU):
        super(SimpleNN, self).__init__()
        # Camada de entrada (2 características) para camada oculta (64 neurônios)
        self.fc1 = nn.Linear(2, 64)
        # Função de ativação (pode ser ReLU, Sigmoid, Tanh, etc.)
        self.activation = activation_fn()
        # Camada oculta (64 neurônios) para camada de saída (1 neurônio para classificação binária)
        self.fc2 = nn.Linear(64, 1)
        # Sigmoid para a camada de saída para produzir probabilidades entre 0 e 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.activation(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# 3. Função de Treinamento
def train_model(model, X_train, y_train, X_test, y_test, criterion, optimizer_class, lr, num_epochs, batch_size):
    optimizer = optimizer_class(model.parameters(), lr=lr)
    train_losses = []
    test_losses = []

    # Criar DataLoader para batching
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        model.train() # Define o modelo para modo de treinamento
        current_train_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            current_train_loss += loss.item() * inputs.size(0) # Acumula perda ponderada pelo tamanho do batch

        train_loss_avg = current_train_loss / len(train_dataset)
        train_losses.append(train_loss_avg)

        # Avaliar no conjunto de teste
        model.eval() # Define o modelo para modo de avaliação
        with torch.no_grad(): # Desativa o cálculo de gradientes
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test).item()
            test_losses.append(test_loss)

        if (epoch + 1) % (num_epochs // 10) == 0:
            print(f'  Época [{epoch+1}/{num_epochs}], Perda de Treinamento: {train_loss_avg:.4f}, Perda de Teste: {test_loss:.4f}')
    return train_losses, test_losses

# 4. Variação de Hiperparâmetros e Visualização

# Hiperparâmetros base
base_lr = 0.01
base_epochs = 1000
base_batch_size = 32
base_optimizer = optim.Adam
base_activation = nn.ReLU
base_criterion = nn.BCELoss() # Binary Cross-Entropy Loss para classificação binária

# Função para plotar os resultados
def plot_results(title, train_losses, test_losses, model, X_test, y_test):
    plt.figure(figsize=(15, 6))

    # Plotar as perdas
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Perda de Treinamento')
    plt.plot(test_losses, label='Perda de Teste')
    plt.title(f'Perda ao longo das Épocas - {title}')
    plt.xlabel('Época')
    plt.ylabel('Perda')
    plt.legend()
    plt.grid(True)

    # Plotar a fronteira de decisão
    plt.subplot(1, 2, 2)
    # Criar um grid de pontos para prever
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    grid_points = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        Z = model(grid_points).numpy().reshape(xx.shape)
        Z = (Z > 0.5).astype(int) # Converte probabilidades para classes (0 ou 1)

    plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')
    plt.scatter(X_train[:, 0].numpy(), X_train[:, 1].numpy(), c=y_train.numpy().flatten(), cmap='viridis', edgecolors='k', s=20, label='Treinamento')
    plt.scatter(X_test[:, 0].numpy(), X_test[:, 1].numpy(), c=y_test.numpy().flatten(), cmap='plasma', marker='x', s=100, label='Teste')
    plt.title(f'Fronteira de Decisão - {title}')
    plt.xlabel('Característica 1')
    plt.ylabel('Característica 2')
    plt.legend()
    plt.grid(True)
    plt.show()

# --- Variação da Taxa de Aprendizado (Learning Rate) ---
print("\n--- Testando diferentes Taxas de Aprendizado (Learning Rate) ---")
learning_rates = [0.1, 0.01, 0.001]
for lr in learning_rates:
    print(f"  Treinando com LR = {lr}")
    model = SimpleNN(activation_fn=base_activation)
    train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test,
                                            base_criterion, base_optimizer, lr, base_epochs, base_batch_size)
    plot_results(f'LR = {lr}', train_losses, test_losses, model, X_test, y_test)

# --- Variação do Número de Épocas ---
print("\n--- Testando diferentes Números de Épocas ---")
epochs_list = [200, 1000, 3000]
for epochs in epochs_list:
    print(f"  Treinando com {epochs} Épocas")
    model = SimpleNN(activation_fn=base_activation)
    train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test,
                                            base_criterion, base_optimizer, base_lr, epochs, base_batch_size)
    plot_results(f'Épocas = {epochs}', train_losses, test_losses, model, X_test, y_test)

# --- Variação do Tamanho do Batch (Batch Size) ---
print("\n--- Testando diferentes Tamanhos de Batch ---")
batch_sizes = [1, 32, 64, len(X_train)] # 1 (SGD puro), 32 (comum), 64, tamanho total (Batch Gradient Descent)
for bs in batch_sizes:
    print(f"  Treinando com Batch Size = {bs}")
    model = SimpleNN(activation_fn=base_activation)
    train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test,
                                            base_criterion, base_optimizer, base_lr, base_epochs, bs)
    plot_results(f'Batch Size = {bs}', train_losses, test_losses, model, X_test, y_test)

# --- Variação do Otimizador ---
print("\n--- Testando diferentes Otimizadores ---")
optimizers = {'SGD': optim.SGD, 'Adam': optim.Adam}
for name, opt_class in optimizers.items():
    print(f"  Treinando com Otimizador = {name}")
    model = SimpleNN(activation_fn=base_activation)
    train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test,
                                            base_criterion, opt_class, base_lr, base_epochs, base_batch_size)
    plot_results(f'Otimizador = {name}', train_losses, test_losses, model, X_test, y_test)

# --- Variação da Função de Ativação ---
print("\n--- Testando diferentes Funções de Ativação ---")
activations = {'ReLU': nn.ReLU, 'Sigmoid': nn.Sigmoid, 'Tanh': nn.Tanh}
for name, act_fn in activations.items():
    print(f"  Treinando com Função de Ativação = {name}")
    model = SimpleNN(activation_fn=act_fn)
    train_losses, test_losses = train_model(model, X_train, y_train, X_test, y_test,
                                            base_criterion, base_optimizer, base_lr, base_epochs, base_batch_size)
    plot_results(f'Função de Ativação = {name}', train_losses, test_losses, model, X_test, y_test)

print("\n--- Exemplos de Variação de Hiperparâmetros Concluídos ---")


Output hidden; open in https://colab.research.google.com to view.